In [1]:
import os
import networkx as nx
import numpy as np
import re
import glob

####### Global options #######
import configuration

pi = np.pi

vname_dict = {'V':1,'Er':2,'Ti':3,'Ce':4,'S':5,
			  'H':6,'He':7,'Li':8,'Be':9,'B':10,
			  'C':11,'N':12,'O':13,'F':14,'Ne':15,
			  'Na':16,'Mg':17,'Al':18,'Si':19,'P':20 ,
			  'Cl':21,'Ar':22,'K':23,'Ca':24,'Sc':24,
			  'Cr':26,'Mn':27,'Fe':28,'Co':29,'Ni':30}

metal_elements = ['Ac','Ag','Al','Am','Au','Ba','Be','Bi',
				  'Bk','Ca','Cd','Ce','Cf','Cm','Co','Cr',
				  'Cs','Cu','Dy','Er','Es','Eu','Fe','Fm',
				  'Ga','Gd','Hf','Hg','Ho','In','Ir',
				  'K','La','Li','Lr','Lu','Md','Mg','Mn',
				  'Mo','Na','Nb','Nd','Ni','No','Np','Os',
				  'Pa','Pb','Pd','Pm','Pr','Pt','Pu','Ra',
				  'Rb','Re','Rh','Ru','Sc','Sm','Sn','Sr',
				  'Ta','Tb','Tc','Th','Ti','Tl','Tm','U',
				  'V','W','Y','Yb','Zn','Zr']


####### Global options #######
IGNORE_ALL_ERRORS = configuration.IGNORE_ALL_ERRORS
#PRINT = configuration.PRINT
PRINT =True
CONNECTION_SITE_BOND_LENGTH = configuration.CONNECTION_SITE_BOND_LENGTH
WRITE_CHECK_FILES = configuration.WRITE_CHECK_FILES
WRITE_CIF = configuration.WRITE_CIF
ALL_NODE_COMBINATIONS = configuration.ALL_NODE_COMBINATIONS
USER_SPECIFIED_NODE_ASSIGNMENT = configuration.USER_SPECIFIED_NODE_ASSIGNMENT
COMBINATORIAL_EDGE_ASSIGNMENT = configuration.COMBINATORIAL_EDGE_ASSIGNMENT
#CHARGES = configuration.CHARGES
CHARGES = False
SCALING_ITERATIONS = configuration.SCALING_ITERATIONS
SYMMETRY_TOL = configuration.SYMMETRY_TOL
BOND_TOL = configuration.BOND_TOL
ORIENTATION_DEPENDENT_NODES = configuration.ORIENTATION_DEPENDENT_NODES
PLACE_EDGES_BETWEEN_CONNECTION_POINTS = configuration.PLACE_EDGES_BETWEEN_CONNECTION_POINTS
RECORD_CALLBACK = configuration.RECORD_CALLBACK
OUTPUT_SCALING_DATA = configuration.OUTPUT_SCALING_DATA
FIX_UC = configuration.FIX_UC
MIN_CELL_LENGTH = configuration.MIN_CELL_LENGTH
OPT_METHOD = configuration.OPT_METHOD
PRE_SCALE = configuration.PRE_SCALE
SINGLE_METAL_MOFS_ONLY = configuration.SINGLE_METAL_MOFS_ONLY
MOFS_ONLY = configuration.MOFS_ONLY
MERGE_CATENATED_NETS = configuration.MERGE_CATENATED_NETS
RUN_PARALLEL = configuration.RUN_PARALLEL
REMOVE_DUMMY_ATOMS = configuration.REMOVE_DUMMY_ATOMS

from ciftemplate2graph import ct2g
from vertex_edge_assign import vertex_assign, assign_node_vecs2edges
from cycle_cocyle import cycle_cocyle, Bstar_alpha
from bbcif_properties import cncalc, bbelems
from SBU_geometry import SBU_coords
from scale import scale
from scaled_embedding2coords import omega2coords
from place_bbs import scaled_node_and_edge_vectors, place_nodes_tetra, place_edges
from remove_net_charge import fix_charges
from remove_dummy_atoms import remove_Fr
from adjust_edges import adjust_edges
from write_cifs import write_check_cif, write_cif_nobond, bond_connected_components, distance_search_bond, fix_bond_sym, merge_catenated_cifs
from scale_animation import scaling_callback_animation, write_scaling_callback_animation, animate_objective_minimization
import itertools
from random import choice
from ditopic import *
from supercell import Carte_points_generator
from output import temp_xyz,tempgro
from terminations import *

In [2]:
# ct2g(template.CIF)
# SG, start, unit_cell, set(cns), set(e_types), cifname, aL, bL, cL, alpha, beta, gamma, max_le, catenation

#@TG template graph
#@start fractional coordinate
#@unit_cell Cartesian coordinate
#@TVT Template Vertex Table(the degree (number of connections) of node n in the main graph G, node type)
#@TET template ((l[0],l[1]))) edge connected node pair (only numeric part of the node identifier)
#@TNAME template name
#@catenation if G has seperated (connected) subgraph

## import matplotlib.pyplot as pl
## G1 = nx.path_graph(4)
## nx.add_path(G1, [10, 11, 12])
## nx.add_path(G1, [9, 11, 13])
## l = [len(c) for c in sorted(nx.connected_components(G1), key=len, reverse=True)]
## print(l,sorted(nx.connected_components(G1)))
## pl.figure()
## nx.draw_networkx(G1)
## pl.show()

In [3]:
template = 'she.cif'
supercell= np.array([1,1,1])
box_bound= supercell+1
#template = 'fcu.cif' #UIO6x

templates_dir = 'template_database'
#template = 'rna.cif'
#nodes_dir = 'nodesdatabase'
nodes_dir = 'nodes'
edges_dir = 'edges'


In [4]:

PLACE_EDGES_BETWEEN_CONNECTION_POINTS = True
PRINT=False
print()
print('=========================================================================================================')
print('template :',template)                                          
print('=========================================================================================================')
print()
	
cat_count = 0
for net in ct2g(template,templates_dir):

		cat_count += 1
		TG, start, unit_cell, TVT, TET, TNAME, a, b, c, ang_alpha, ang_beta, ang_gamma, max_le, catenation = net

		TVT = sorted(TVT, key=lambda x:x[0], reverse=True) # sort node with connected degree, the first one is the highest(full)-coordinated node
		TET = sorted(TET, reverse=True) #sort node_pair by the node_index
		#get node cif information from node dir
		
		node_cns = [(cncalc(node, nodes_dir), node) for node in os.listdir(nodes_dir)]

		print('Number of vertices = ', len(TG.nodes()))
		print('Number of edges = ', len(TG.edges()))
		print()

		edge_counts = dict((data['type'],0) for e0,e1,data in TG.edges(data=True))
		for e0,e1,data in TG.edges(data=True):
			edge_counts[data['type']] += 1
		
		if PRINT:
	
			print('There are', len(TG.nodes()), 'vertices in the voltage graph:')
			print()
			v = 0
	
			for node in TG.nodes():
				v += 1
				print(v,':',node)
				node_dict = TG.nodes[node]
				print('type : ', node_dict['type'])
				print('cartesian coords : ', node_dict['ccoords'])
				print('fractional coords : ', node_dict['fcoords'])
				#print('degree : ', node_dict['cn'][0])
				print()
	
			print('There are', len(TG.edges()), 'edges in the voltage graph:')
			print()
	
			for edge in TG.edges(data=True,keys=True):
				edge_dict = edge[3]
				ind = edge[2]
				print(ind,':',edge[0],edge[1])
				print('length : ',edge_dict['length'])
				print('type : ',edge_dict['type'])
				print('label : ',edge_dict['label'])
				print('positive direction :',edge_dict['pd'])
				print('cartesian coords : ',edge_dict['ccoords'])
				print('fractional coords : ',edge_dict['fcoords'])
				print()
	
		vas = vertex_assign(nodes_dir,TG, TVT, node_cns, unit_cell, USER_SPECIFIED_NODE_ASSIGNMENT, SYMMETRY_TOL, ALL_NODE_COMBINATIONS)
		CB,CO = cycle_cocyle(TG)

		for va in vas:
			if len(va) == 0:
				print('At least one vertex does not have a building block with the correct number of connection sites.')
				print('Moving to the next template...')
				print()
				continue
	
		if len(CB) != (len(TG.edges()) - len(TG.nodes()) + 1):
			print('The cycle basis is incorrect.')
			print('The number of cycles in the cycle basis does not equal the rank of the cycle space.')
			print('Moving to the next template...')
			continue
		
		num_edges = len(TG.edges())
		Bstar, alpha = Bstar_alpha(CB,CO,TG,num_edges)

		if PRINT:
			print('B* (top) and alpha (bottom) for the barycentric embedding are:')
			print()
			for i in Bstar:
				print(i)
			print()
			for i in alpha:
				print(i)
			print()
	
		num_vertices = len(TG.nodes())
	
		if COMBINATORIAL_EDGE_ASSIGNMENT:
			eas = list(itertools.product([e for e in os.listdir('edges')], repeat = len(TET)))
		else:
			edge_files = sorted([e for e in os.listdir('edges')])
			eas = []
			i = 0
			while len(eas) < len(TET):
				eas.append(edge_files[i])
				i += 1
				if i == len(edge_files):
					i = 0
			eas = [eas]
	
		g = 0

		for va in vas:
			#check if assigned node has metal element 
			node_elems = [bbelems(i[1], nodes_dir) for i in va]
			metals = [[i for i in j if i in metal_elements] for j in node_elems]
			metals = list(set([i for j in metals for i in j]))
			#set node cif files as vertex assignment
			v_set0 = [('v' + str(vname_dict[re.sub('[0-9]','',i[0])]), i[1]) for i in va]
			v_set1 = sorted(list(set(v_set0)), key=lambda x: x[0])
			v_set = [v[0] + '-' + v[1] for v in v_set1]
	
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print('vertex assignment : ',v_set)
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print()

			if SINGLE_METAL_MOFS_ONLY and len(metals) != 1:
				print(v_set, 'contains no metals or multiple metal elements, no cif will be written')
				print()
				continue

			if MOFS_ONLY and len(metals) < 1:
				print(v_set, 'contains no metals, no cif will be written')
				print()
				continue
			
			# add cifname to TG.nodes
			for v in va:
				for n in TG.nodes(data=True):
					if v[0] == n[0]:
						n[1]['cifname'] = v[1]
			
			for ea in eas:
	
				g += 1
	
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print('edge assignment : ',ea)
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print()
				
				type_assign = dict((k,[]) for k in sorted(TET, reverse=True))
				for k,m in zip(TET,ea):
					type_assign[k] = m
				
				# add cifname to TG.edge
				for e in TG.edges(data=True):
					ty = e[2]['type']
					for k in type_assign:
						if ty == k or (ty[1],ty[0]) == k:
							e[2]['cifname'] = type_assign[k]

				num_possible_XX_bonds = 0
				for edge_type, cifname in zip(TET, ea):
					if cifname == 'ntn_edge.cif':
						factor = 1
					else:
						factor = 2
					edge_type_count = edge_counts[edge_type]
					num_possible_XX_bonds += factor * edge_type_count

				ea_dict = assign_node_vecs2edges(nodes_dir,TG, unit_cell, SYMMETRY_TOL, template)
				all_SBU_coords = SBU_coords(TG, ea_dict, CONNECTION_SITE_BOND_LENGTH)
				sc_a, sc_b, sc_c, sc_alpha, sc_beta, sc_gamma, sc_covar, Bstar_inv, max_length, callbackresults, ncra, ncca, scaling_data = scale(all_SBU_coords,a,b,c,ang_alpha,ang_beta,ang_gamma,max_le,num_vertices,Bstar,alpha,num_edges,FIX_UC,SCALING_ITERATIONS,PRE_SCALE,MIN_CELL_LENGTH,OPT_METHOD)
		
				print('*******************************************')
				print('The scaled unit cell parameters are : ')
				print('*******************************************')
				print('a    :', np.round(sc_a, 5))
				print('b    :', np.round(sc_b, 5))
				print('c    :', np.round(sc_c, 5))
				print('alpha:', np.round(sc_alpha, 5))
				print('beta :', np.round(sc_beta, 5))
				print('gamma:', np.round(sc_gamma, 5))
				print()
	
				for sc, name in zip((sc_a, sc_b, sc_c), ('a', 'b', 'c')):
					cflag = False
					if sc == MIN_CELL_LENGTH:
						print('unit cell parameter', name, 'may have collapsed during scaling!')
						print('try re-running with', name, 'fixed or a larger MIN_CELL_LENGTH')
						print('no cif will be written')
						cflag = True
	
				if cflag:
					continue
	
				scaled_params = [sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma]
			
				sc_Alpha = np.r_[alpha[0:num_edges-num_vertices+1,:], sc_covar]
				sc_omega_plus = np.dot(Bstar_inv, sc_Alpha)
			
				ax = sc_a
				ay = 0.0
				az = 0.0
				bx = sc_b * np.cos(sc_gamma * pi/180.0)
				by = sc_b * np.sin(sc_gamma * pi/180.0)
				bz = 0.0
				cx = sc_c * np.cos(sc_beta * pi/180.0)
				cy = (sc_c * sc_b * np.cos(sc_alpha * pi/180.0) - bx * cx) / by
				cz = (sc_c ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
				sc_unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
				
				scaled_coords = omega2coords(start, TG, sc_omega_plus, (sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma), num_vertices,templates_dir, template, g, WRITE_CHECK_FILES)
				nvecs,evecs,node_placed_edges = scaled_node_and_edge_vectors(scaled_coords, sc_omega_plus, sc_unit_cell, ea_dict)
				placed_nodes, porphyrin_node,frame_nbb_node,porphyrin_node_name,node_bonds = place_nodes_tetra(nvecs, nodes_dir,CHARGES, ORIENTATION_DEPENDENT_NODES)
				placed_edges, edge_bonds = place_edges(evecs, edges_dir,CHARGES, len(placed_nodes))
				#print(f"place_edges{place_edges}")
	
				if RECORD_CALLBACK:
	
					vnames = '_'.join([v.split('.')[0] for v in v_set])
	
					if len(ea) <= 5:
						enames = '_'.join([e[0:-4] for e in ea])
					else:
						enames = str(len(ea)) + '_edges'
	
					prefix = template[0:-4] + '_' +  vnames + '_' + enames
	
					frames = scaling_callback_animation(callbackresults, alpha, Bstar_inv, ncra, ncca, num_vertices, num_edges, TG, template, g, False)
					write_scaling_callback_animation(frames, prefix)
					animate_objective_minimization(callbackresults, prefix)
	
				if PLACE_EDGES_BETWEEN_CONNECTION_POINTS:
					placed_edges,cleaved_placed_edges,cleaved_placed_nodes,X_Opair = adjust_edges(placed_edges, placed_nodes, sc_unit_cell)
				
				# add classifination 
				#cleaved_placed_edges = placed_OXedges
				cleaved_placed_nodes = np.c_[cleaved_placed_nodes, np.array(['NODE' for i in range(len(cleaved_placed_nodes))])]
				cleaved_placed_edges = np.c_[cleaved_placed_edges, np.array(['EDGE' for i in range(len(cleaved_placed_edges))])]
				placed_nodes = np.c_[placed_nodes, np.array(['node' for i in range(len(placed_nodes))])]
				placed_edges = np.c_[placed_edges, np.array(['edge' for i in range(len(placed_edges))])]
				

				placed_all = list(placed_nodes) + list(placed_edges)
				bonds_all = node_bonds + edge_bonds
		
				if WRITE_CHECK_FILES:
					write_check_cif(template, placed_nodes, placed_edges, g, scaled_params, sc_unit_cell)
			
				if REMOVE_DUMMY_ATOMS:
					placed_all, bonds_all, nconnections = remove_Fr(placed_all,bonds_all)
				
				print('computing X-X bonds...')
				print()
				print('*******************************************')
				print('Bond formation : ')
				print('*******************************************')
				
				#fixed_bonds, nbcount, bond_check_passed = bond_connected_components(placed_all, bonds_all, sc_unit_cell, max_length, BOND_TOL, nconnections, num_possible_XX_bonds)
				#print('there were ', nbcount, ' X-X bonds formed')
				#bond_check_passed =False
				#if bond_check_passed:
				#	print('bond check passed')
				#	bond_check_code = ''
				#else:
				#	print('bond check failed, attempting distance search bonding...')
				#	fixed_bonds, nbcount = distance_search_bond(placed_all, bonds_all, sc_unit_cell, 2.5)
				#	bond_check_code = '_BOND_CHECK_FAILED'
				#	print('there were', nbcount, 'X-X bonds formed')
				#print()
		
				if CHARGES:
					fc_placed_all, netcharge, onetcharge, rcb = fix_charges(placed_all)
				else:
					fc_placed_all = placed_all
			
				fc_placed_all = placed_all
				#fixed_bonds = fix_bond_sym(fixed_bonds, placed_all, sc_unit_cell)
	
				if CHARGES:
					print('*******************************************')
					print('Charge information :                       ')
					print('*******************************************')
					print('old net charge                  :', np.round(onetcharge, 5))
					print('rescaling magnitude             :', np.round(rcb, 5))
			
					remove_net = choice(range(len(fc_placed_all)))
					fc_placed_all[remove_net][4] -= np.round(netcharge, 4)
			
					print('new net charge (after rescaling):', np.sum([li[4] for li in fc_placed_all]))
					print()

				vnames = '_'.join([v.split('.')[0] for v in v_set])
				enames_list = [e[0:-4] for e in ea]
				enames_grouped = [list(edge_gr) for ind,edge_gr in itertools.groupby(enames_list)]
				enames_grouped = [(len(edge_gr), list(set(edge_gr))) for edge_gr in enames_grouped]
				enames_flat = [str(L) + '-' + '_'.join(names) for L,names in enames_grouped]
				enames = '_'.join(enames_flat)
				bond_check_code = 'nobond'
				if catenation:
					outcifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '_' + 'CAT' + str(cat_count) + '.cif'
				else:
					outcifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '.cif'
		
				if WRITE_CIF:
					print('writing cif...')
					print()
					if len(cifname) > 255:
						cifname = cifname[0:241]+'_truncated.cif'
					write_cif_nobond(fc_placed_all, scaled_params, sc_unit_cell, outcifname, CHARGES, wrap_coords=False)

if catenation and MERGE_CATENATED_NETS:
	
	print('merging catenated cifs...')
	cat_cifs = glob.glob('output_cifs/*_CAT*.cif')

	for comb in itertools.combinations(cat_cifs, cat_count):

		builds = [name[0:-9] for name in comb]

		print(set(builds))

		if len(set(builds)) == 1:
			pass
		else:
			continue

		merge_catenated_cifs(comb, CHARGES)

	#for cif in cat_cifs:
	#	os.remove(cif)


template : she.cif

Number of vertices =  20
Number of edges =  48

*****************************************************************
RMSD of the compatible node BBs with assigned vertices:          
*****************************************************************

vertex Er (6 connected)
     6c_Zr_1_Ch.cif deviation = 0.35973 (within tolerance)
     MIL-53.cif deviation = 0.59159 (outside tolerance)
* 1 compatible building blocks out of 2 available for node Er *
vertex V (4 connected)
     4c_porphyrin_Ch.cif deviation = 0.00381 (within tolerance)
* 1 compatible building blocks out of 1 available for node V *

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
vertex assignment :  ['v1-4c_porphyrin_Ch.cif', 'v2-6c_Zr_1_Ch.cif']
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
edge assignment :  ('1B_2F_Ch

<class 'networkx.utils.decorators.argmap'> compilation 30:3: FutureWarning: 

shortest_path will return an iterator that yields
(node, path) pairs instead of a dictionary when source
and target are unspecified beginning in version 3.5

To keep the current behavior, use:

	dict(nx.shortest_path(G))


anf, aff_all, chg, all_inds, [bbind] * len(anf)(['C937', 'C938', 'C939', 'C940', 'C941', 'C942', 'F943', 'H944', 'H945', 'F946'], array([[ 4.99241243,  8.79726223, 14.39097029],
       [ 5.21004445,  9.08930828, 15.74593073],
       [ 6.53545385,  9.30411624, 16.15817227],
       [ 7.59813164,  9.23324279, 15.26997049],
       [ 7.3805206 ,  8.94101028, 13.91507925],
       [ 6.05511119,  8.72620231, 13.50283771],
       [ 5.77012356,  8.44195742, 12.22052521],
       [ 3.98837429,  8.62354531, 14.02381526],
       [ 8.60219075,  9.40677325, 15.63719472],
       [ 6.82044149,  9.58836114, 17.44048478]]), ['-0.210094', '-0.090513', '0.298055', '-0.210097', '-0.090649', '0.298129', '-0.195863', '0.181779', '0.181846', '-0.195753'], array(['C1', 'X2', 'C3', 'C4', 'X5', 'C6', 'FG7', 'H8', 'H9', 'FG10'],
      dtype='<U4'), [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2])
anf, aff_all, chg, all_inds, [bbind] * len(anf)(['C947', 'C948', 'C949', 'C950', 'C951', 'C952', 'F953', 'H954', 'H955', 'F956'

In [5]:
def limit_x(x):
    while x>0.5:
        x=x-1
    while x< -0.5:
        x = x+1
    return x
def centerize_edges_cc(target_edges_list,target_node_c,sc_unit_cell):
    edges_update=[]
    edges_update_append = edges_update.append
    for te_ccord in target_edges_list:
        te = te_ccord[:,1:4]- target_node_c
        te_fvec = np.dot(te,np.linalg.inv(sc_unit_cell))
        edge_c_fvec = np.mean(te_fvec,axis=0).tolist()
        cx,cy,cz = edge_c_fvec
        cx1 = limit_x(cx)
        cy1 = limit_x(cy)
        cz1 = limit_x(cz)
        differ = np.asarray([cx1,cy1,cz1])-np.asarray(edge_c_fvec)
        te_update = np.hstack((te_ccord[:,0:1],te+np.dot(differ,sc_unit_cell)+target_node_c,te_ccord[:,4:]))
        edges_update_append(te_update)
        print(differ)
    return np.vstack((edges_update))

def centerize_edges_fc(target_edges_list,target_node_c_fc):
    edges_update=[]
    edges_update_append = edges_update.append
    for te_fcord in target_edges_list:
        te_fvec = te_fcord[:,1:4]- target_node_c_fc
        edge_c_fvec = np.mean(te_fvec,axis=0).tolist()
        cx,cy,cz = edge_c_fvec
        cx1 = limit_x(cx)
        cy1 = limit_x(cy)
        cz1 = limit_x(cz)
        differ = np.asarray([cx1,cy1,cz1])-np.asarray(edge_c_fvec)
        te_update = np.hstack((te_fcord[:,0:1],te_fvec+differ+target_node_c_fc,te_fcord[:,4:]))
        edges_update_append(te_update)
        print(differ)
    return np.vstack((edges_update))

def merge_porphyrin_node_edge(TG,porphyrin_node_name,placed_nodes_arr,placed_edges_arr,sc_unit_cell):
    porphyrins=[]
    porphyrins_append = porphyrins.append
    edges_dict_list=list(TG.edges(data=True,keys=True))
    for pcnode in porphyrin_node_name:
        pcnode_idx=TG.nodes[pcnode]['index']
        linked_porphyrin = []
        linked_porphyrin_append= linked_porphyrin.append
        for i in range(len(edges_dict_list)):
            e_dict=edges_dict_list[i]
            check = (pcnode in e_dict[0:2])
            if check:
                linked_porphyrin_append(e_dict[2][0])
        porphyrins_append((pcnode,pcnode_idx,linked_porphyrin))

    porphyrin_edges = []
    porphyrin_edges_append = porphyrin_edges.append
    for i in range(len(porphyrins)):
        pcnode = porphyrins[i]
        node_idx = [pcnode[1]]
        linked_p_idx = pcnode[2]
        target_nodes = fetch_node_withidx(placed_nodes_arr,node_idx)
        target_nodes_c = np.mean(target_nodes[:,1:4],axis=0)
        target_nodes_fc = np.dot(target_nodes_c,np.linalg.inv(sc_unit_cell))
        moded_trans_fc = np.mod(target_nodes_fc,1) - target_nodes_fc
        target_nodes[:,1:4] = target_nodes[:,1:4] + np.dot(moded_trans_fc,sc_unit_cell)

        target_edge_list = fetch_edge_withidx_sep(placed_edges_arr,linked_p_idx)
        
        target_nodes_c = np.mean(target_nodes[:,1:4],axis=0)
        target_edges = centerize_edges_cc(target_edge_list,target_nodes_c,sc_unit_cell)
    
        porphyrin_edge = np.vstack((target_nodes,target_edges))
        porphyrin_edge[:,6]=[-1*i-2]*len(porphyrin_edge)
        porphyrin_edge[:,7]=['EDGE']*len(porphyrin_edge)
        #fvec=np.dot(porphyrin_edge[:,1:4],np.linalg.inv(sc_unit_cell))
        #fvec=np.mod(fvec,1)
        #porphyrin_edge[:,1:4] = np.dot(fvec,sc_unit_cell)
        porphyrin_edges_append(porphyrin_edge)

    porphyrin_edges_ccoords = np.vstack(porphyrin_edges)

    return porphyrin_edges_ccoords

def merge_porphyrin_node_edge_fc(TG,porphyrin_node_name,placed_nodes_arr_fc,placed_edges_arr_fc):
    porphyrins=[]
    porphyrins_append = porphyrins.append
    edges_dict_list=list(TG.edges(data=True,keys=True))
    for pcnode in porphyrin_node_name:
        pcnode_idx=TG.nodes[pcnode]['index']
        linked_porphyrin = []
        linked_porphyrin_append= linked_porphyrin.append
        for i in range(len(edges_dict_list)):
            e_dict=edges_dict_list[i]
            check = (pcnode in e_dict[0:2])
            if check:
                linked_porphyrin_append(e_dict[2][0])
        porphyrins_append((pcnode,pcnode_idx,linked_porphyrin))

    porphyrin_edges = []
    porphyrin_edges_append = porphyrin_edges.append
    for i in range(len(porphyrins)):
        pcnode = porphyrins[i]
        node_idx = [pcnode[1]]
        linked_p_idx = pcnode[2]
        target_nodes = fetch_node_withidx(placed_nodes_arr_fc,node_idx)
        target_nodes_c_fc = np.mean(target_nodes[:,1:4],axis=0)
        moded_trans_fc = np.mod(target_nodes_c_fc,1) - target_nodes_c_fc
        target_nodes[:,1:4] = target_nodes[:,1:4] + moded_trans_fc

        target_edge_list = fetch_edge_withidx_sep(placed_edges_arr_fc,linked_p_idx)
        
        target_nodes_c_fc = np.mean(target_nodes[:,1:4],axis=0)
        target_edges = centerize_edges_fc(target_edge_list,target_nodes_c_fc)
    
        porphyrin_edge = np.vstack((target_nodes,target_edges))
        porphyrin_edge[:,6]=[-1*i-2]*len(porphyrin_edge)
        porphyrin_edge[:,7]=['EDGE']*len(porphyrin_edge)
        #fvec=np.dot(porphyrin_edge[:,1:4],np.linalg.inv(sc_unit_cell))
        #fvec=np.mod(fvec,1)
        #porphyrin_edge[:,1:4] = np.dot(fvec,sc_unit_cell)
        porphyrin_edges_append(porphyrin_edge)

    porphyrin_edges_fcoords = np.vstack(porphyrin_edges)

    return porphyrin_edges_fcoords

In [6]:
frame_node_name=[i for i in list(TG.nodes()) if i not in porphyrin_node_name]

frame_node_fc=np.asarray([TG.nodes[fn]['fcoords']for fn in frame_node_name])

In [7]:
def find_new_node_beginning(arr):
    #find min x,y,z see if [x,y,z] in
    #if not in then find minx then find miny then find minz 
    min_xyz=np.min(arr,axis=0)
    if any(np.array_equal(row, min_xyz) for row in arr):
        return min_xyz
    else:
        min_x = np.min(arr[:,0])
        min_x_rows = arr[arr[:,0]==min_x]
        min_xyz = np.min(min_x_rows,axis=0)
        if  any(np.array_equal(row, min_xyz) for row in arr):
            return min_xyz
        else:
            min_y = np.min(min_x_rows[:,1])
            min_xy_rows = min_x_rows[min_x_rows[:,1]==min_y]
            min_xyz = np.min(min_xy_rows,axis=0)
            return min_xyz

new_beginning_fc = find_new_node_beginning(frame_node_fc)

In [8]:

placed_nodes_arr,nodes_id=placed_arr(cleaved_placed_nodes)
placed_edges_arr,edges_id=placed_arr(cleaved_placed_edges)
placed_nodes_fc = np.hstack((placed_nodes_arr[:,0:1],np.dot(placed_nodes_arr[:,1:4],np.linalg.inv(sc_unit_cell))-new_beginning_fc,placed_nodes_arr[:,4:]))
placed_edges_fc = np.hstack((placed_edges_arr[:,0:1],np.dot(placed_edges_arr[:,1:4],np.linalg.inv(sc_unit_cell))-new_beginning_fc,placed_edges_arr[:,4:]))
placed_nodes_arr[:,1:4]-=np.dot(new_beginning_fc,sc_unit_cell)
placed_edges_arr[:,1:4]-=np.dot(new_beginning_fc,sc_unit_cell)

frame_node_ccoords= np.c_[frame_nbb_node,['NODE']*len(frame_nbb_node)]
placed_frame_node,_ = placed_arr(frame_node_ccoords)
placed_frame_node_fc = np.hstack((placed_frame_node[:,0:1],np.dot(placed_frame_node[:,1:4],np.linalg.inv(sc_unit_cell))-new_beginning_fc,placed_frame_node[:,4:]))
placed_frame_node[:,1:4]-=np.dot(new_beginning_fc,sc_unit_cell)


#porphyrin_edges_ccoords = merge_porphyrin_node_edge(TG,porphyrin_node_name,placed_nodes_arr,placed_edges_arr,sc_unit_cell)
porphyrin_edges_fcoords = merge_porphyrin_node_edge_fc(TG,porphyrin_node_name,placed_nodes_fc,placed_edges_fc)
target_all_fc = np.vstack((placed_frame_node_fc,porphyrin_edges_fcoords))
#target_all = np.vstack((placed_frame_node,porphyrin_edges_ccoords))
supercell_Carte = Carte_points_generator(supercell)
#target_all[:,1:4] = target_all[:,1:4]-np.dot(new_beginning_fc,sc_unit_cell)

[1. 1. 0.]
[1. 1. 0.]
[1. 1. 0.]
[1. 1. 0.]
[0. 1. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 1.]
[1. 0. 0.]
[1. 0. 1.]
[1. 0. 1.]
[0. 0. 0.]
[0. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
[0. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
[0. 1. 1.]
[0. 1. 0.]
[0. 1. 1.]
[0. 1. 0.]
[0. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 1.]


In [9]:
def outxyz(output,all_array):
    atoms_number = len(all_array)
    newxyz = []
    with open(output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):
            row = all_array[i]
            value_label = row[0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = float(row[1])  # x
            value_y = float(row[2])  # y
            value_z = float(row[3])  # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)
outxyz('1.xyz',porphyrin_edges_fcoords)

In [10]:
porphyrin_edges_fc = porphyrin_edges_fcoords
#porphyrin_edges_fvec = np.dot(porphyrin_edges_ccoords[:,1:4],np.linalg.inv(sc_unit_cell))
#porphyrin_edges_fc=np.hstack((porphyrin_edges_ccoords[:,0:1],porphyrin_edges_fvec,porphyrin_edges_ccoords[:,4:]))

In [11]:
#nodes_idx =np.arange(1,1+len(TG.nodes()),1).tolist()
#edges_idx =np.arange(1,1+len(TG.edges()),1).tolist()
#
#
##frame=Frame(supercell,sc_unit_cell)
##frame.supercell_generator()
#supercell_Carte = Carte_points_generator(supercell)
#
#t_nodes_idx = nodes_idx#[3,5,7,8]
#t_edges_idx = edges_idx
#target_nodes = fetch_node_withidx(placed_nodes_arr,t_nodes_idx)
#target_edges = fetch_edge_withidx(placed_edges_arr,t_edges_idx)
#target_all = np.vstack((target_nodes,target_edges))

In [12]:
#s_fvec_all,row_diff_idx = supercell_nodeedge(supercell_Carte,target_all,sc_unit_cell,box_bound)
#safe_res_fc,extra_res_fc,boundary_node_res = filt_boundary_res(s_fvec_all,row_diff_idx,box_bound)
#bare_nodeedge_fc=np.vstack((safe_res_fc,extra_res_fc))

s_fvec_all,row_diff_idx = supercell_nodeedge_fc(supercell_Carte,target_all_fc,box_bound) 
safe_res_fc,extra_res_fc,boundary_node_res = filt_boundary_res(s_fvec_all,row_diff_idx,box_bound)
bare_nodeedge_fc=np.vstack((safe_res_fc,extra_res_fc))

differ res idx{1, 2, 3, 4, 6, 7, 8, 11, 13, 14, 15, 16, 18, 21, 22, 24, 26, 27, 28, 30, 33, 34, 36, 38, 41, 43, 44, 46, 47, 48, 49, 51, 55, 56, 61, 64, 67, 68, 69, 70, 81, 82, 83, 86, 87, 88, 91, 93, 94, 95, 96, 98, 101, 102, 106, 108, 110, 111, 113, 114, 116, 118, 121, 123, 126, 127, 129, 131, 133, 135, 136, 138, 149, 150, 151, 153, 156, 158}
see EDGE11
see EDGE13
see EDGE14
see EDGE15
see EDGE16
see EDGE18
see EDGE30
see EDGE33
see EDGE34
see EDGE36
see EDGE38
see EDGE49
see EDGE51
see EDGE55
see EDGE56
see EDGE69
see EDGE70
see EDGE91
see EDGE93
see EDGE94
see EDGE95
see EDGE96
see EDGE98
see EDGE110
see EDGE111
see EDGE113
see EDGE114
see EDGE116
see EDGE118
see EDGE129
see EDGE131
see EDGE133
see EDGE135
see EDGE136
see EDGE138
see EDGE149
see EDGE150
see EDGE151
see EDGE153
see EDGE156
see EDGE158


In [13]:
term_file = 'methyl.pdb'
ex_node_cxo_cc = exposed_Xs_Os_boundary_node(boundary_node_res,bare_nodeedge_fc,sc_unit_cell,box_bound)

In [14]:
terms = add_terminations(term_file,ex_node_cxo_cc)
terms_array = np.vstack((terms))
bare_nodeedge_array = np.hstack((bare_nodeedge_fc[:,:-3],np.dot(bare_nodeedge_fc[:,-3:],sc_unit_cell)))
node_edge_term_array = np.vstack((bare_nodeedge_array,terms_array))

temp_xyz("0term_supercell.xyz",node_edge_term_array)
temp_xyz("1term_supercell.xyz",bare_nodeedge_array)
tempgro('1te.gro',bare_nodeedge_array)
tempgro('0te.gro',node_edge_term_array)

In [15]:
bare_nodeedge_array.shape

(17658, 9)

In [21]:
scalar = 0.98
s_fvec_all_loose,row_diff_idx_loose = supercell_nodeedge_fc_loose_check(supercell_Carte,target_all_fc,box_bound,scalar) 
safe_res_fc_loose,extra_res_fc_loose,boundary_node_res_loose = filt_boundary_res_loose_check(s_fvec_all,row_diff_idx,box_bound,scalar)
bare_nodeedge_fc_loose=np.vstack((safe_res_fc_loose,extra_res_fc_loose))

term_file = 'methyl.pdb'
ex_node_cxo_cc_loose = exposed_Xs_Os_boundary_node(boundary_node_res_loose,bare_nodeedge_fc_loose,sc_unit_cell,box_bound)
terms_loose = add_terminations(term_file,ex_node_cxo_cc_loose)
terms_array_loose = np.vstack((terms_loose))
bare_nodeedge_array_loose = np.hstack((bare_nodeedge_fc_loose[:,:-3],np.dot(bare_nodeedge_fc_loose[:,-3:],sc_unit_cell)))
node_edge_term_array_loose = np.vstack((bare_nodeedge_array_loose,terms_array_loose))

#temp_xyz("0term_supercell.xyz",node_edge_term_array_loose)
#temp_xyz("1term_supercell.xyz",bare_nodeedge_array_loose)
tempgro('11te.gro',bare_nodeedge_array_loose)
tempgro('01te.gro',node_edge_term_array_loose)

differ res idx{1, 2, 3, 4, 6, 7, 8, 11, 13, 14, 15, 16, 18, 21, 22, 24, 26, 27, 28, 30, 33, 34, 36, 38, 41, 43, 44, 46, 47, 48, 49, 51, 55, 56, 61, 64, 67, 68, 69, 70, 81, 82, 83, 86, 87, 88, 91, 93, 94, 95, 96, 98, 101, 102, 106, 108, 110, 111, 113, 114, 116, 118, 121, 123, 126, 127, 129, 131, 133, 135, 136, 138, 149, 150, 151, 153, 156, 158}
11 [0.0, 0.0, 2.0] [0.8 0.4 2.3]
13 [0.0, 2.0, 0.0] [1.  2.2 0.8]
14 [0.0, 2.0, 0.0] [0.5 2.2 0.7]
15 [0.0, 0.0, 2.0] [0.7 0.6 2.2]
16 [0.0, 0.0, 2.0] [0.8 0.2 2. ]
in 16 [0.0, 0.0, 2.0]
16 [0.0, 2.0, 0.0] [0.8 2.2 0. ]
16 [0.0, 2.0, 2.0] [0.8 2.2 2. ]
18 [0.0, 2.0, 0.0] [0.8 2.2 0.5]
30 [-0.0, -0.0, -2.0] [ 0.3  0.7 -0.1]
33 [0.0, 2.0, 0.0] [1.  2.2 1.8]
34 [0.0, 2.0, 0.0] [0.5 2.2 1.7]
36 [0.0, 2.0, 0.0] [0.8 2.2 1. ]
38 [0.0, 2.0, 0.0] [0.8 2.2 1.5]
49 [-0.0, -2.0, -0.0] [ 0.8 -0.   0.3]
in 49 [-0.0, -2.0, -0.0]
51 [0.0, 0.0, 2.0] [0.8 1.4 2.3]
55 [0.0, 0.0, 2.0] [0.7 1.6 2.2]
56 [0.0, 0.0, 2.0] [0.8 1.2 2. ]
in 56 [0.0, 0.0, 2.0]
69 [-0.0, -2

In [22]:
bare_nodeedge_array_loose.shape

(18834, 9)